In [1]:
from llama_cpp import Llama
import numpy as np
from matplotlib import pyplot as plt
import ctypes
import torch
from torch.nn import functional as F

In [2]:
model_bin = "/media/captdishwasher/Samshmung/horenbergerb/llama/llama.cpp/models/llama/converted/llama_13b_ggml_q4_3.bin"
lora_bin = "/media/captdishwasher/Samshmung/horenbergerb/llama/llama.cpp/models/SuperCOT-LoRA/13b/ggml/cutoff-2048/ggml-adapter-model.bin"

LLM = Llama(model_path=model_bin, lora_path=lora_bin, n_ctx=2048, n_threads=7, use_mmap=False, n_batch=512)

llama.cpp: loading model from /media/captdishwasher/Samshmung/horenbergerb/llama/llama.cpp/models/llama/converted/llama_13b_ggml_q4_3.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 6 (mostly Q4_3)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size = 9534493.73 KB
llama_model_load_internal: mem required  = 11359.03 MB (+ 1608.00 MB per state)
....................................................................................................
llama_init_from_file: kv self size  = 1600.00 MB
llama_apply_lora_fro

In [3]:
def get_logits_and_probs(prompt):
    tokens = LLM.tokenize(prompt.encode('utf-8'))
    LLM.reset()
    LLM.eval(tokens)
    logits = LLM._logits()
    logits = np.array(logits[0])
    probs=F.softmax(torch.from_numpy(logits)).numpy()
    return logits, probs

In [4]:
def plot_logits(logits, bins=50):
    counts, bins = np.histogram(logits, bins=bins)
    plt.stairs(counts, bins)
    plt.title('Histogram of logits')

In [5]:
def plot_probs(probs, bins=1000, bins_to_skip=2):
    counts, bins = np.histogram(probs, bins=bins)
    # Observing that the vast majority of tokens are in the first bin, i.e. have incredibly small probability.
    print('There are {} tokens outside the {} smallest bins.'.format(len(np.where(probs>bins[bins_to_skip])[0]), bins_to_skip))
    # Don't plot the smallest bins since they are not interesting and dwarf all other bins
    plt.stairs(counts[bins_to_skip:], bins[bins_to_skip:])
    plt.title('Histogram of Token Probabilities')

In [6]:
def plot_most_likely_tokens(probs, num_tokens_to_plot=10):
    ind = np.argpartition(probs, -num_tokens_to_plot)[-num_tokens_to_plot:]

    likely_tokens = []
    token_probs = []

    for token in ind:
        c_token = [(ctypes.c_int)(*[token])]
        print('Token: {}, Prob: {}'.format(LLM.detokenize(c_token), probs[token]))
        # print('Token: {} Detokenized: {}'.format(token, LLM.detokenize(c_token)))
        likely_tokens.append(LLM.detokenize(c_token))
        token_probs.append(probs[token])

    plt.bar(likely_tokens, token_probs)
    plt.title('Bar chart of 10 most likely tokens')

In [1]:
#command = "Don't do anything, John!"
#command = "John, beegle freegle, would ya?"
#command = "John, set the heading to 150 mark 190."
#command = "Bring us around, John! Reverse course!"
#command = "Bring us to full stop, John."
#command = "Full stop, John"
#command = "I love jelly!"
#command = "John, I love jelly!"
command = "Set the jelly, John!"
command = "Jelly set bongus heading."

prompt = """### Instruction:
You are to act as a verbal command parser on the bridge of a star ship from the Star Trek series.
You will be given a command spoken by the captain of the ship. 

You must determine which command is meant by the input. The possible outputs are [SET HEADING, SET SPEED, UNKNOWN]

If the command is nonsensical, unclear, or null, then output \"UNKNOWN\".

Respond with the command most appropriate command for the given input.

### Input:
{}

### Response:
""".format(command)
print(prompt)
logits, probs = get_logits_and_probs(prompt)
plot_logits(logits=logits)
plt.show()
plot_probs(probs)
plt.show()
plot_most_likely_tokens(probs)
plt.show()

### Instruction:
You are to act as a verbal command parser on the bridge of a star ship from the Star Trek series.
You will be given a command spoken by the captain of the ship. 

You must determine which command is meant by the input. The possible outputs are [SET HEADING, SET SPEED, REVERSE COURSE, UNKNOWN]

If the command is nonsensical, unclear, or null, then output "UNKNOWN".

Respond with the command most appropriate command for the given input.

### Input:
Jelly set bongus heading.

### Response:



NameError: name 'get_logits_and_probs' is not defined